In [1]:
import os
from scripts.MicroBrix import MicroBrix
import geopandas as gpd
import json

In [2]:
from numpy import mean
import random

In [3]:
def indicator(geogrid, geogrid_data):
    layers = []

    #HEATMAP
    features = []
    for cell in geogrid['features']:
        feature = {}
        lat,lon = zip(*cell['geometry']['coordinates'][0])
        lat,lon = mean(lat),mean(lon)
        feature['coordinates'] = [lat,lon]
        feature['weight'] = random.random()*10
        features.append(feature)

    layers.append({"id":"heatmapN1","type":"heatmap","data":features,"properties":{"colorRange":[[237,248,251],[204,236,230],[153,216,201],[102,194,164],[44,162,95],[0,109,44]]}})

    #HEATMAP
    features = []
    for cell in geogrid['features']:
        feature = {}
        lat,lon = zip(*cell['geometry']['coordinates'][0])
        lat,lon = mean(lat),mean(lon)
        feature['coordinates'] = [lat,lon]
        feature['weight'] = random.random()*10
        features.append(feature)

    layers.append({"id":"heatmapN2","type":"heatmap","data":features,"properties":{}})

    point_a = random.choice(geogrid['features'])
    point_b = random.choice(geogrid['features'])
    lat_a,lon_a = zip(*point_a['geometry']['coordinates'][0])
    lat_a,lon_a = mean(lat_a),mean(lon_a)
    lat_b,lon_b = zip(*point_b['geometry']['coordinates'][0])
    lat_b,lon_b = mean(lat_b),mean(lon_b)

    #ARC
    arc_data = [{
        'inbound': 72633,
        'outbound': 74735,
        'from': {
            'name': 'Point A',
            'coordinates': [lat_a,lon_a]
        },
        'to': {
            'name': 'Point B',
            'coordinates': [lat_b,lon_b]
        }
    }]

    layers.append({"id": "arcN1", "type":"arc", "data":arc_data,"properties":{"width":2}})

    # Numeric indicators
    numeric = []

    numeric.append({"viz_type": "bar", "name": "example1", "value": random.random(), "description": "An example"})
    numeric.append({"viz_type": "bar", "name": "example2", "value": random.random(), "description": "An example 2"})
    numeric.append({"viz_type": "bar", "name": "example3", "value": random.random(), "description": "An example 3"})
    numeric.append({"viz_type": "bar", "name": "example4", "value": random.random(), "description": "An example 4"})

    numeric.append({"viz_type": "radar", "name": "exampleRadar1", "value": random.random(), "description": "An example"})
    numeric.append({"viz_type": "radar", "name": "exampleRadar2", "value": random.random(), "description": "An example 2"})
    numeric.append({"viz_type": "radar", "name": "exampleRadar3", "value": random.random(), "description": "An example 3"})
    numeric.append({"viz_type": "radar", "name": "exampleRadar4", "value": random.random(), "description": "An example 4"})

    return layers, numeric

In [12]:
def read_land_uses():
    from shapely.geometry import Polygon
    data = gpd.read_parquet('/app/data/landuses_actual.parquet')

    # Convertir la proyección a EPSG:4326
    gdf = data.to_crs(epsg=4326)

    # Verificar si el GeoDataFrame tiene geometría
    if 'geometry' not in gdf.columns:
        raise ValueError("El GeoDataFrame debe contener una columna 'geometry' con la geometría de los polígonos.")


    # Convertir los polígonos a coordenadas (x, y)
    gdf['geometry'] = gdf['geometry'].apply(lambda geom: Polygon([(x, y) for x, y, _ in geom.exterior.coords]))
        
    features = json.loads(gdf.to_json())
    return features

def polygon_indicator(geogrid, geogrid_data):
    layers = []
    numeric = []
    features = read_land_uses()
    print(features)
    layers.append({"id": "land_uses", "type":"geojson", "data": features,"properties":{"filled": True}})
    return layers, numeric

In [8]:
gdf = gdf[['Uso', 'geometry']]

In [10]:
gdf.to_parquet('/app/data/land_uses_clean.parquet')

In [11]:
gdf

,Uso,geometry
0,SITIO ERIAZO,"POLYGON ((-73.07318 -36.82560, -73.07391 -36.8..."
1,SITIO ERIAZO,"POLYGON ((-73.06750 -36.82260, -73.06746 -36.8..."
2,BIENES COMUNES,"POLYGON ((-73.07173 -36.82608, -73.07173 -36.8..."
3,BIENES COMUNES,"POLYGON ((-73.07190 -36.82598, -73.07192 -36.8..."
4,BIENES COMUNES,"POLYGON ((-73.07090 -36.82481, -73.07055 -36.8..."
...,...,...
3129,MULTI ROL,"POLYGON ((-73.06159 -36.82806, -73.06191 -36.8..."
3130,PRE ROL,"POLYGON ((-73.06132 -36.82795, -73.06159 -36.8..."
3131,MULTI ROL,"POLYGON ((-73.06062 -36.82814, -73.06070 -36.8..."
3132,MULTI ROL,"POLYGON ((-73.06062 -36.82814, -73.06087 -36.8..."


In [17]:
connection = MicroBrix(table_name='test', module_function=polygon_indicator, keep_updating=False, quietly=True)
connection.listen()

## Opened connection
{'type': 'FeatureCollection', 'features': [{'id': '0', 'type': 'Feature', 'properties': {'Id': 1, 'Uso': 'SITIO ERIAZO', 'area_predio': 27346.344287460925, 'plate_id': 0.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-73.07318120673156, -36.82559668855182], [-73.07390806293043, -36.825179212249104], [-73.07441697092986, -36.82475820079252], [-73.07414924198483, -36.82461077701814], [-73.07400824731978, -36.82461445476079], [-73.07387675680357, -36.82453481914863], [-73.07371441989045, -36.82450237891522], [-73.07355387645252, -36.824398424861954], [-73.0733992522418, -36.82435418557663], [-73.07325383404094, -36.824238552654805], [-73.07314219226002, -36.82425462790344], [-73.07292115091354, -36.82419738768661], [-73.07290826877845, -36.82411967390712], [-73.07266379307248, -36.82410974868867], [-73.07250901936978, -36.82407146719457], [-73.07242156042783, -36.82401042989441], [-73.07236967058934, -36.82400958839845], [-73.07229778502169, -36.823918994048405

In [18]:
connection.stop()

In [24]:


gdf = convertir_proyeccion_y_poligono(data)

In [26]:
gdf

,Id,Uso,area_predio,plate_id,geometry
0,1,SITIO ERIAZO,27346.344287,0.0,"POLYGON ((-73.07318 -36.82560, -73.07391 -36.8..."
1,2,SITIO ERIAZO,10492.919632,0.0,"POLYGON ((-73.06750 -36.82260, -73.06746 -36.8..."
2,3,BIENES COMUNES,2466.609968,0.0,"POLYGON ((-73.07173 -36.82608, -73.07173 -36.8..."
3,4,BIENES COMUNES,2107.678619,0.0,"POLYGON ((-73.07190 -36.82598, -73.07192 -36.8..."
4,5,BIENES COMUNES,756.003752,0.0,"POLYGON ((-73.07090 -36.82481, -73.07055 -36.8..."
...,...,...,...,...,...
3129,4133,MULTI ROL,1528.206919,0.0,"POLYGON ((-73.06159 -36.82806, -73.06191 -36.8..."
3130,4134,PRE ROL,1327.066841,0.0,"POLYGON ((-73.06132 -36.82795, -73.06159 -36.8..."
3131,4150,MULTI ROL,1683.256411,0.0,"POLYGON ((-73.06062 -36.82814, -73.06070 -36.8..."
3132,4151,MULTI ROL,1544.466189,0.0,"POLYGON ((-73.06062 -36.82814, -73.06087 -36.8..."
